In [9]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [11]:
# Lee los casos de area de influencia y deja los diagnosticos en formato codigo
casos_area_de_influencia = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="casos_area_de_influencia_INT"
)
casos_area_de_influencia["Diagnostico"] = (
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0]
)
casos_area_de_influencia = casos_area_de_influencia.set_index("Diagnostico")

# Lee los porcentajes de quirurgicos y deja los diagnosticos en formato codigo
porcentajes_de_quirurgicos = pd.read_excel(
    "../data/external/incidencias_y_prevalencias_INT.xlsx", sheet_name="porcentajes_trazadoras"
)
porcentajes_de_quirurgicos["Diagnostico"] = (
    porcentajes_de_quirurgicos["Diagnostico"].str.split(" - ").str[0]
)
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos.set_index("Diagnostico")
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos["Porcentaje Quirúrgico"]
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos.dropna()

In [12]:
DIAGS_AREA_DE_INFLUENCIA = casos_area_de_influencia.index

En este caso, se multiplicaran los casos del area de influencia por el porcentaje de intervenciones quirúrgicas.

In [13]:
casos_quirurgicos = casos_area_de_influencia[COLUMNAS_POBLACION_INE].mul(
    porcentajes_de_quirurgicos, axis=0
)

In [14]:
duraciones_int_q = pd.read_excel(
    "../data/raw/5_duracion_int_q/duraciones_de_int_q_por_diagnostico.xlsx"
)

duraciones_int_q["diag_01_principal_(cod)"] = (
    duraciones_int_q["diag_01_principal_(cod)"].str.replace(".", "", regex=False).str.ljust(4, "X")
)

duraciones_int_q["diag_01_principal_(cod)"] = duraciones_int_q[
    "diag_01_principal_(cod)"
].str.replace("C341", "C34N")

duraciones_int_q = duraciones_int_q.query(
    "`diag_01_principal_(cod)`.isin(@DIAGS_AREA_DE_INFLUENCIA)"
)
duraciones_int_q = duraciones_int_q.set_index("diag_01_principal_(cod)")

# Convierte las duraciones a tiempo
COLUMNAS_DURACION = ["mean", "std", "min", "25%", "50%", "75%", "max"]
duraciones_int_q[COLUMNAS_DURACION] = duraciones_int_q[COLUMNAS_DURACION].apply(
    lambda x: pd.to_timedelta(x, unit="D")
)

duraciones_int_q = duraciones_int_q.query("ano_de_egreso == 2019")["mean"]

In [51]:
# Multiplica la cantidad de tiempo promedio por diagnostico con los casos quirurgicos
tiempo_utilizado_pabellon = casos_quirurgicos.mul(duraciones_int_q, axis=0)
# Convierte a horas el tiempo necesario por pabellon
tiempo_utilizado_pabellon_horas = tiempo_utilizado_pabellon.apply(
    lambda x: x.dt.total_seconds() / 3600
)

In [63]:
# Obtiene la cantidad de dias laborales por anio
cantidad_dias_laborales = build_features.obtener_cantidad_de_dias_laborales_por_anio(
    "01-01-2017", "01-01-2036"
)

# Obtiene la cantidad de horas laborales por anio
cantidad_dias_laborales.index = cantidad_dias_laborales.index.year.astype(str)
CANTIDAD_HORAS_DE_UN_PABELLON_POR_DIA = 12 # De 8AM a 8PM
horas_laborales = cantidad_dias_laborales * CANTIDAD_HORAS_DE_UN_PABELLON_POR_DIA

In [67]:
cantidad_de_pabellones_necesarios = tiempo_utilizado_pabellon_horas.div(horas_laborales, axis=1)

In [68]:
cantidad_de_pabellones_necesarios

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
A152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C34G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C34N,0.814933,0.836791,0.850062,0.859459,0.874265,0.884822,0.901632,0.900031,0.905427,0.903337,0.908224,0.923904,0.935877,0.932476,0.92891,0.921501,0.932217,0.950479,0.957259
C34S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
